In [169]:
import json

dataset = "wc"

data_file = f"{dataset}_merged.json"

data = []

with open(data_file) as f:
    for line in f:
        data.append(json.loads(line))

data = [
    x for x in data
    if "N/A" not in x["task_type"] and "unknown" not in x["task_type"]
]

for x in data:
    x["dataset"] = dataset

In [13]:

# data_file = f"sample_{dataset}.json"

# with open(data_file) as f:
#     data = json.load(f)


In [170]:
from bertopic import BERTopic

def add_labels(dataset_type, column_name, data):
    topics_path = f"{dataset}_topic_model_{dataset_type}/topics.json"

    with open(topics_path) as f:
        topics = json.load(f)

    model_path = f"{dataset}_topic_model_{dataset_type}"
    loaded_model = BERTopic.load(model_path)
    topic_info = loaded_model.get_topic_info()

    print(topic_info)

    topic_list = topics["topics"]

    other_topics = topic_info[topic_info["Name"].str.contains("_")]["Topic"]

    if len(other_topics) > 0:
        other_topic = other_topics.iloc[0]
    else:
        other_topic = len(topic_info)

    print(other_topic)

    topic_list = [min(x, other_topic) for x in topic_list]
    topic_names = topic_info["Name"][:other_topic].tolist()
    topic_names += ["other"]

    for x, y in zip(data, topic_list):
        x[column_name] = topic_names[y]

    

In [171]:
add_labels("2", "clusterized_task_type", data)

    Topic  Count                                               Name  \
0       0    484                                    code generation   
1       1    373                                           analysis   
2       2    313                                 information search   
3       3    299                                    article writing   
4       4    298                                  technical support   
5       5    297                                   creative writing   
6       6    291                                concept explanation   
7       7    206                                           planning   
8       8    206                                          debugging   
9       9    185                                        translation   
10     10    177                                      brainstorming   
11     11    176                                             advice   
12     12    174                                             design   
13    

In [172]:
add_labels("topics", "clusterized_domain", data)

    Topic  Count           Name  \
0       0    804    programming   
1       1    564     technology   
2       2    368       business   
3       3    285       language   
4       4    240      education   
5       5    192     literature   
6       6    187      technical   
7       7    155      marketing   
8       8    155         health   
9       9    146        finance   
10     10    144  entertainment   
11     11    133             AI   
12     12    112        science   
13     13    101     psychology   
14     14     98         gaming   
15     15     94       creative   
16     16     90    mathematics   
17     17     82  communication   
18     18     71            law   
19     19     67         travel   
20     20     58       politics   
21     21     56     philosophy   
22     22     52   social media   
23     23     41         sports   
24     24     39        history   

                                       Representation  Representative_Docs  
0           

In [173]:
add_labels("reasoning", "clusterized_reasoning", data)

    Topic  Count                                               Name  \
0       0    737                                    problem-solving   
1       1    678                                  analytical skills   
2       2    427                                         creativity   
3       3    415                                  logical reasoning   
4       4    403                               language proficiency   
5       5    376                                   domain knowledge   
6       6    270                                  critical thinking   
7       7    241                               communication skills   
8       8    206                              information synthesis   
9       9    194                                       storytelling   
10     10    137                                           planning   
11     11    124                                     advisory skill   
12     12    108                                            empathy   
13    

In [174]:
data

[{'id': '0',
  'domain': 'math',
  'summary': 'Find the derivative of a given function.',
  'task_type': 'math problem solving',
  'reasoning': 'mathematical reasoning',
  'dataset': 'sg',
  'clusterized_task_type': 'calculation',
  'clusterized_domain': 'mathematics',
  'clusterized_reasoning': 'logical reasoning'},
 {'id': '1',
  'domain': 'education',
  'summary': 'Create flashcards based on a provided text.',
  'task_type': 'educational material creation',
  'reasoning': 'comprehension, information retrieval, summarization, simplification',
  'dataset': 'sg',
  'clusterized_task_type': 'design',
  'clusterized_domain': 'education',
  'clusterized_reasoning': 'information synthesis'},
 {'id': '2',
  'domain': 'technology',
  'summary': 'Access a text file and create flashcards based on the content.',
  'task_type': 'file access, flashcard creation',
  'reasoning': 'comprehension, information retrieval, summarization, simplification, creativity',
  'dataset': 'sg',
  'clusterized_tas

In [175]:
import pandas as pd

In [176]:
df = pd.DataFrame(data)

In [61]:
df.columns

Index(['id', 'domain', 'summary', 'task_type', 'reasoning', 'dataset',
       'clusterized_task_type', 'clusterized_domain', 'clusterized_reasoning'],
      dtype='object')

In [177]:
n_added = 3

cols = df.columns.tolist()
print(cols)
cols = cols[:1] + cols[-n_added-1:] + cols[1:-n_added-1]
print(cols)
cols = cols[:3+n_added] + cols[4+n_added:] + [cols[n_added+3]]
cols

['id', 'domain', 'summary', 'task_type', 'reasoning', 'dataset', 'clusterized_task_type', 'clusterized_domain', 'clusterized_reasoning']
['id', 'dataset', 'clusterized_task_type', 'clusterized_domain', 'clusterized_reasoning', 'domain', 'summary', 'task_type', 'reasoning']


['id',
 'dataset',
 'clusterized_task_type',
 'clusterized_domain',
 'clusterized_reasoning',
 'domain',
 'task_type',
 'reasoning',
 'summary']

In [178]:
df = df[cols]

In [179]:
df.to_csv(f"combined_{dataset}.csv", index=False)

In [156]:
datasets = ["wc", "sg", "quora"]


df = pd.read_csv(f"combined_{datasets[0]}.csv")

for dataset in datasets[1:]:
    df = pd.concat([df, pd.read_csv(f"combined_{dataset}.csv")])

df.to_csv("combined_all.csv", index=False)
df

,id,dataset,clusterized_task_type,clusterized_domain,clusterized_reasoning,domain,task_type,reasoning,summary
0,0,wc,creative writing,creative,creativity,"creative writing, paranormal activity","creative writing, story development","imagination, plot development, creative proble...",Write a story about 2 brothers starting a smal...
1,1,wc,creative writing,gaming,storytelling,"fantasy, gaming","dialogue writing, character development","character development, plot development, world...",Create a dialogue and scene for a fantasy game.
2,2,wc,article writing,personal,communication skills,"personal information, debate","personal information sharing, persuasive writing","argumentation, persuasive writing, public spea...",Provide personal information and write a debat...
3,3,wc,calculation,science,analytical skills,chemistry,"stoichiometry, calculation","mathematical calculation, stoichiometry, unit...",Calculate the number of moles of AlCl3 in 10 g...
4,4,wc,creative writing,creative,storytelling,creative writing,"creative writing, storytelling","imagination, storytelling, descriptive langua...",Write a scene in first person perspective base...
...,...,...,...,...,...,...,...,...,...
4728,4993,quora,NaN,technology,problem-solving,"technology, audio extraction","software recommendation, technical advice","technical knowledge, problem-solving, researc...",Seek information on extracting audio from vid...
4729,4994,quora,NaN,education,planning,"education, career planning","academic/career advice, study planning","strategic planning, academic knowledge, caree...",Request guidance on preparing for the IAS exa...
4730,4995,quora,NaN,personal development,critical thinking,"personal, life planning","life planning, philosophical discussion","critical thinking, philosophical reasoning, d...",Question about life expectancy and planning.
4731,4996,quora,NaN,politics,critical thinking,"political, environmental","political analysis, climate change discussion","critical thinking, political analysis, enviro...",Question about the beliefs of Donald Trump re...


In [9]:
topic_list

[4,
 4,
 3,
 4,
 0,
 3,
 3,
 0,
 1,
 1,
 1,
 5,
 0,
 7,
 5,
 2,
 8,
 2,
 1,
 0,
 0,
 5,
 0,
 2,
 6,
 3,
 0,
 0,
 1,
 4,
 0,
 6,
 2,
 0,
 7,
 1,
 2,
 2,
 1,
 0,
 7,
 1,
 5,
 5,
 4,
 0,
 1,
 2,
 0,
 7,
 1,
 6,
 1,
 0,
 0,
 4,
 1,
 2,
 6,
 2,
 6,
 5,
 2,
 1,
 5,
 1,
 6,
 4,
 7,
 0,
 9,
 5,
 9,
 1,
 1,
 0,
 1,
 5,
 0,
 1,
 5,
 2,
 10,
 0,
 0,
 7,
 0,
 0,
 3,
 7,
 5,
 10,
 0,
 0,
 8,
 1,
 5,
 5,
 2,
 8,
 6,
 4,
 2,
 2,
 1,
 0,
 0,
 1,
 3,
 0,
 4,
 0,
 3,
 2,
 7,
 2,
 0,
 4,
 1,
 7,
 1,
 1,
 0,
 5,
 0,
 0,
 7,
 3,
 3,
 6,
 2,
 2,
 1,
 3,
 6,
 6,
 0,
 4,
 1,
 0,
 4,
 5,
 0,
 7,
 4,
 0,
 2,
 0,
 2,
 1,
 3,
 0,
 1,
 4,
 5,
 0,
 4,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 6,
 3,
 1,
 0,
 4,
 3,
 2,
 8,
 7,
 4,
 0,
 3,
 2,
 6,
 7,
 10,
 2,
 7,
 6,
 4,
 4,
 10,
 5,
 4,
 6,
 3,
 6,
 3,
 0,
 0,
 7,
 3,
 9,
 0,
 9,
 0,
 9,
 0,
 1,
 5,
 2,
 2,
 4,
 1,
 0,
 3,
 0,
 0,
 1,
 1,
 8,
 1,
 2,
 4,
 0,
 2,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 5,
 5,
 3,
 4,
 0,
 4,
 10,
 2,
 4,
 1,
 1,
 9,
 9,
 8,
 0,
 1,
 5,
 2,
 8,
 6,
 2,
 4,

In [11]:
topic_names = [topic_info["Name"][x] for x in topic_list]

In [15]:
len(topic_list)

12652

In [12]:
len(topic_names), len(data)

(12652, 5000)

In [8]:
filtered_data = [x for t, x in zip(topic_list, data) if t == 1]

In [9]:
filtered_data[:100]

[{'conversations': [{'from': 'human',
    'value': 'How do l stop intrusive thoughts?'}],
  'id': '8'},
 {'conversations': [{'from': 'human',
    'value': 'On the American stock market, if a price says 768, does that mean $7.68 or $768?'}],
  'id': '9'},
 {'conversations': [{'from': 'human',
    'value': 'What are the best job opportunities for someone who has BSc petroleum engineering plus a MBA degree to work in Norway?'}],
  'id': '10'},
 {'conversations': [{'from': 'human', 'value': 'What is my face shape?'}],
  'id': '18'},
 {'conversations': [{'from': 'human',
    'value': 'Why should we have a girl friend?'}],
  'id': '28'},
 {'conversations': [{'from': 'human',
    'value': 'Why do people think just because you are attractive you have a high self-esteem?'}],
  'id': '35'},
 {'conversations': [{'from': 'human', 'value': 'What is a square?'}],
  'id': '38'},
 {'conversations': [{'from': 'human',
    'value': 'How hard is the A+ certification exam?'}],
  'id': '41'},
 {'conversati

In [45]:
combined = [x.strip() for y in filtered_data for x in y["domain"].split(",")]

In [46]:
from collections import Counter
counts = Counter(combined)

counts

Counter({'technology': 62,
         'education': 37,
         'psychology': 30,
         'finance': 25,
         'politics': 22,
         'health': 19,
         'entertainment': 17,
         'business': 15,
         'history': 14,
         'physics': 12,
         'career': 12,
         'relationships': 12,
         'language': 12,
         'science': 11,
         'social media': 10,
         'biology': 9,
         'travel': 8,
         'relationship': 8,
         'literature': 8,
         'mathematics': 8,
         'philosophy': 7,
         'programming': 7,
         'economics': 7,
         'culture': 7,
         'sociology': 7,
         'engineering': 6,
         'gaming': 6,
         'chemistry': 6,
         'healthcare': 6,
         'employment': 6,
         'medical': 6,
         'social': 5,
         'religion': 5,
         'legal': 5,
         'international relations': 5,
         'social issues': 5,
         'geography': 5,
         'personal': 4,
         'film': 4,
         